In [1]:
from tpot import TPOTClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

In [2]:
digits = load_digits()
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target,
                                                    train_size=0.75, test_size=0.25)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1347, 64), (450, 64), (1347,), (450,))

In [3]:
tpot = TPOTClassifier(verbosity=2, max_time_mins=5, population_size=40)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))

                                                                            
5.12 minutes have elapsed. TPOT will close down.                            
TPOT closed during evaluation in one generation.
                                                                            
                                                                            
TPOT closed prematurely. Will use the current best pipeline.
                                                                            
Best pipeline: MLPClassifier(SelectFwe(input_matrix, alpha=0.045), alpha=0.1, learning_rate_init=0.001)
0.9644444444444444


In [17]:
tpot.export('tpot_digits_pipeline.py')

In [6]:
# save sklearn datasets to parquet
import pandas as pd
from sklearn.datasets import load_digits
digits = load_digits()
df = pd.DataFrame(digits.data, columns=digits.feature_names)
df['target'] = digits.target
df.to_parquet('digits.parquet')


In [20]:
# %load tpot_digits_pipeline.py
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

# NOTE: Make sure that the outcome column is labeled 'target' in the data file
# Load parquet file into dataframe
tpot_data = pd.read_parquet('digits.parquet')
#tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
features = tpot_data.drop('target', axis=1)
training_features, testing_features, training_classes, testing_classes = \
            train_test_split(features, tpot_data['target'], random_state=None)

exported_pipeline = KNeighborsClassifier(n_neighbors=4, p=2, weights="distance")

exported_pipeline.fit(training_features, training_classes)
results = exported_pipeline.predict(testing_features)
print(exported_pipeline.score(testing_features, testing_classes))
print(results)

0.9911111111111112
[5 0 5 7 6 1 1 2 2 8 1 5 2 6 7 0 2 6 3 3 1 5 3 4 2 0 8 4 8 5 9 0 3 8 3 3 9
 2 2 1 3 2 6 9 8 3 4 7 6 0 8 5 5 4 9 1 5 6 3 4 6 5 2 1 6 8 4 4 2 1 0 9 1 0
 3 1 7 3 8 3 4 0 7 9 8 5 2 0 2 0 2 3 7 2 4 4 6 2 4 8 2 0 8 4 6 2 8 7 7 6 6
 4 5 5 0 5 2 3 4 6 1 7 3 9 4 4 8 4 4 5 6 1 0 6 5 8 2 6 1 2 9 0 9 8 3 8 8 8
 3 2 8 8 2 0 1 3 8 6 2 9 4 2 9 3 5 5 5 3 3 6 2 1 9 5 0 1 5 5 5 6 6 2 0 4 5
 0 1 7 9 3 0 9 9 6 5 8 7 8 8 6 4 5 4 5 1 5 3 9 5 9 5 5 7 4 1 5 5 4 1 7 3 2
 2 4 9 2 7 2 6 1 8 5 8 8 4 4 7 4 1 3 9 6 6 8 0 2 8 6 7 6 7 7 8 4 3 5 2 3 0
 8 4 8 0 5 7 6 3 2 4 4 0 7 9 2 0 8 9 7 2 8 4 3 4 7 5 7 6 9 9 0 4 0 5 6 8 5
 5 5 1 9 5 6 7 6 4 8 8 8 2 5 0 0 5 2 8 1 8 0 9 4 3 2 5 3 9 9 2 8 5 6 1 0 8
 7 0 2 4 1 5 6 4 5 8 4 5 5 9 3 4 4 9 5 5 8 0 8 4 2 2 1 3 6 3 1 0 0 1 0 2 4
 6 7 7 4 6 6 9 1 1 8 4 6 8 5 6 3 9 6 2 1 1 4 9 9 6 7 0 1 6 7 4 4 2 5 0 0 1
 7 3 3 8 1 6 3 4 4 6 0 6 0 5 1 0 1 9 6 1 1 7 2 4 3 4 5 5 7 7 1 9 8 6 8 3 5
 7 9 9 6 5 2]


In [15]:
# Save the model
import joblib
joblib.dump(exported_pipeline, 'digits_model.joblib')

['digits_model.joblib']